In [182]:
import numpy as np
import pandas as pd

Loading the dataset

In [183]:
df = pd.read_csv("customer_churn_dataset.csv")
df['ContractType'] = df['ContractType'].map({'Monthly': 0, 'One year': 1, 'Two year': 2})
print(df)

     Age  MonthlyCharges  ContractType  Tenure  Churn
0     56          183.36             0      27      1
1     69           64.87             1      66      0
2     46           93.87             1       5      0
3     32          156.00             1      29      1
4     60           61.18             2      37      0
..   ...             ...           ...     ...    ...
195   66           35.36             1      56      0
196   69           29.30             0      63      0
197   78          115.64             1      48      0
198   49          117.31             0      61      0
199   21          134.74             1      26      1

[200 rows x 5 columns]


Splitting the dataset

In [184]:
x = df.drop(['Churn'], axis=1)
y = df['Churn']

training_split = 0.6
validation_split = 0.2
test_split = 0.2

x_train = x[:int(len(x)*training_split)]
y_train = y[:int(len(y)*training_split)]

x_val = x[int(len(x)*training_split):int(len(x)*training_split)+int(len(x)*validation_split)]
y_val = y[int(len(y)*training_split):int(len(y)*training_split)+int(len(y)*validation_split)]

x_test = x[int(len(x)*training_split)+int(len(x)*validation_split):]
y_test = y[int(len(y)*training_split)+int(len(y)*validation_split):]

print("Training data x : ", x_train.shape)
print(x_train)
print("Training data y : ", y_train.shape)
print(y_train)

print("Validation data x : ", x_val.shape)
print(x_val)
print("Validation data y : ", y_val.shape)
print(y_val)

print("Test data x : ", x_test.shape)
print(x_test)
print("Test data y : ", y_test.shape)
print(y_test)

Training data x :  (120, 4)
     Age  MonthlyCharges  ContractType  Tenure
0     56          183.36             0      27
1     69           64.87             1      66
2     46           93.87             1       5
3     32          156.00             1      29
4     60           61.18             2      37
..   ...             ...           ...     ...
115   33          172.86             1      45
116   62          138.37             0       4
117   35          122.30             2      62
118   64           36.86             0      65
119   70           86.19             0      32

[120 rows x 4 columns]
Training data y :  (120,)
0      1
1      0
2      0
3      1
4      0
      ..
115    0
116    0
117    0
118    0
119    1
Name: Churn, Length: 120, dtype: int64
Validation data x :  (40, 4)
     Age  MonthlyCharges  ContractType  Tenure
120   41           67.74             0      34
121   43           63.92             0      39
122   42          195.14             0      26
123

Normalizing the parameters


In [185]:
x_train = (x_train - x_train.mean()) / x_train.std()
x_val = (x_val - x_val.mean()) / x_val.std()
x_test = (x_test - x_test.mean()) / x_test.std()

print("Normalized Training data x : ", x_train.shape)
print(x_train)
print("Normalized Validation data x : ", x_val.shape)
print(x_val)
print("Normalized Test data x : ", x_test.shape)
print(x_test)

Normalized Training data x :  (120, 4)
          Age  MonthlyCharges  ContractType    Tenure
0    0.293785        1.422782     -1.117401 -0.387601
1    0.974773       -0.826954      0.147581  1.490218
2   -0.230052       -0.276339      0.147581 -1.446884
3   -0.963423        0.903305      0.147581 -0.291303
4    0.503320       -0.897014      1.412563  0.093891
..        ...             ...           ...       ...
115 -0.911039        1.223421      0.147581  0.479085
116  0.608087        0.568570     -1.117401 -1.495033
117 -0.806272        0.263453      1.412563  1.297621
118  0.712854       -1.358771     -1.117401  1.442069
119  1.027156       -0.422157     -1.117401 -0.146855

[120 rows x 4 columns]
Normalized Validation data x :  (40, 4)
          Age  MonthlyCharges  ContractType    Tenure
120 -0.245404       -0.946820     -0.913994 -0.119482
121 -0.135110       -1.015673     -0.913994  0.129439
122 -0.190257        1.349484     -0.913994 -0.517755
123  1.739889       -0.531899    

Initializing the initial parameters
and using the validation set to set alpha 

In [186]:
#LOG LOSS

def logLoss(y, y_pred):
    return -np.mean(y * np.log(y_pred) + (1 - y) * np.log(1 - y_pred))

In [187]:
np.random.seed(0)
w = np.random.rand(x_train.shape[1])
b = np.random.rand(1)
print("Initial weights : ", w)
print("Initial bias : ", b)

Initial weights :  [0.5488135  0.71518937 0.60276338 0.54488318]
Initial bias :  [0.4236548]


Using gradient descent to minimize the cost function with respect to the weights.

In [188]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))


In [189]:
best_alpha = None
best_val_error = float('inf')
epochs = 1000   
possible_alphas = [0.0001, 0.001, 0.01, 0.1]

for alpha in possible_alphas:
    for epoch in range(epochs):
        z = np.dot(x_train, w) + b
        y_pred = sigmoid(z)
        error = y_train - y_pred
        w = w + alpha * np.dot(x_train.T, error) / len(x_train)
        b = b + alpha * np.mean(error)
        
        z_val = np.dot(x_val, w) + b
        y_pred_val = sigmoid(z_val)
        val_error = np.mean(np.abs(y_val - y_pred_val))
        
        if val_error < best_val_error:
            best_val_error = val_error
            best_alpha = alpha
            
print("Best alpha : ", best_alpha)
print("Best validation error : ", best_val_error)

Best alpha :  0.1
Best validation error :  0.3437291704358101


Evaluating the model performance

In [190]:
# Evaluating the model performance
z_test = np.dot(x_test, w) + b
y_pred_test = sigmoid(z_test)
y_pred_test_binary = np.round(y_pred_test)

accuracy = np.mean(y_test == y_pred_test_binary)
precision = np.sum(y_pred_test_binary[y_test == 1] == 1) / np.sum(y_pred_test_binary == 1)
recall = np.sum(y_pred_test_binary[y_test == 1] == 1) / np.sum(y_test == 1)
f1_score = 2 * (precision * recall) / (precision + recall)

print("Z (test set):", z_test)
print("Sigmoid (test set):", y_pred_test)
print("Actual (test set):", y_test)
print("Predicted (test set):", y_pred_test_binary)
print("Error (test set):", np.mean(np.abs(y_test - y_pred_test_binary)))

confusion_matrix = np.zeros((2, 2), dtype=int)
for i in range(len(y_test)):
    confusion_matrix[int(y_test.iloc[i])][int(y_pred_test_binary[i])] += 1

print("Confusion matrix:")
print(confusion_matrix)


Z (test set): [-1.49199622 -1.36525857 -0.03986745 -0.94012707 -1.00150423 -0.01759316
 -0.16905464  0.64880606 -0.8114227  -0.68161303 -1.51680699 -2.33218139
 -0.06543924 -0.34358718 -1.21794003 -1.43286625 -1.47051233 -0.72131476
 -1.14684303 -0.56867023 -0.71738343 -2.63383151 -1.07180432 -0.13287889
 -2.65209461 -0.33049353 -0.62852184 -1.23093367 -0.57643896 -1.10791321
 -1.77866437 -0.87329207 -0.05968782 -1.0337879  -1.61769154 -2.0347602
 -1.86705241 -1.32242613 -1.30728507 -0.60908384]
Sigmoid (test set): [0.1836223  0.20338698 0.49003446 0.28087467 0.26864577 0.49560182
 0.45783671 0.65674136 0.30758741 0.33590139 0.17993219 0.08849255
 0.48364602 0.41493837 0.22829917 0.19265248 0.18686475 0.32710353
 0.24106619 0.36154372 0.32796943 0.06699257 0.2550601  0.46682907
 0.06586003 0.41812054 0.34784578 0.22601805 0.3597524  0.24826013
 0.14446814 0.29456975 0.48508247 0.2623504  0.16552348 0.11560136
 0.13388315 0.21041493 0.21294151 0.35226821]
Actual (test set): 160    1
161

/tmp/ipykernel_386846/1856113630.py:9: RuntimeWarning: invalid value encountered in scalar divide
  f1_score = 2 * (precision * recall) / (precision + recall)


# Q2

loading the dataset

In [191]:
df = pd.read_csv("customer_churn_dataset2.csv")
print("Before modifying the dataset :")
print(df.head(4))
print()
df['ContractType'] = df['ContractType'].map({'Monthly': 0, 'One year': 1, 'Two year': 2})
df['ChurnType'] = df['ChurnType'].map({'Voluntary Churn': 2, 'Involuntary Churn': 1, 'No Churn': 0})
print("After modifying the dataset :")
print(df.head(4))


Before modifying the dataset :
   Age  MonthlyCharges ContractType  Tenure        ChurnType
0   56          183.36      Monthly      27  Voluntary Churn
1   69           64.87     One year      66         No Churn
2   46           93.87     One year       5  Voluntary Churn
3   32          156.00     One year      29  Voluntary Churn

After modifying the dataset :
   Age  MonthlyCharges  ContractType  Tenure  ChurnType
0   56          183.36             0      27          2
1   69           64.87             1      66          0
2   46           93.87             1       5          2
3   32          156.00             1      29          2


Splitting the dataset to Training, Testing and Validation set

In [192]:
x = df.drop(['ChurnType'], axis=1)
y = df['ChurnType']

training_split = 0.6
validation_split = 0.2
test_split = 0.2

x_train = x[:int(len(x)*training_split)]
y_train = y[:int(len(y)*training_split)]

x_val = x[int(len(x)*training_split):int(len(x)*training_split)+int(len(x)*validation_split)]
y_val = y[int(len(y)*training_split):int(len(y)*training_split)+int(len(y)*validation_split)]

x_test = x[int(len(x)*training_split)+int(len(x)*validation_split):]
y_test = y[int(len(y)*training_split)+int(len(y)*validation_split):]

x_train = np.array(x_train)
y_train = np.array(y_train)
x_val = np.array(x_val)
y_val = np.array(y_val)
x_test = np.array(x_test)
y_test = np.array(y_test)

print("Training data x : ", x_train.shape)
print(x_train)
print("Training data y : ", y_train.shape)
print(y_train)

print("Validation data x : ", x_val.shape)
print(x_val)
print("Validation data y : ", y_val.shape)
print(y_val)

print("Test data x : ", x_test.shape)
print(x_test)
print("Test data y : ", y_test.shape)
print(y_test)

Training data x :  (120, 4)
[[ 56.   183.36   0.    27.  ]
 [ 69.    64.87   1.    66.  ]
 [ 46.    93.87   1.     5.  ]
 [ 32.   156.     1.    29.  ]
 [ 60.    61.18   2.    37.  ]
 [ 25.    33.86   0.    38.  ]
 [ 78.    72.16   2.     8.  ]
 [ 38.    49.02   2.    65.  ]
 [ 56.   187.35   1.    17.  ]
 [ 75.   165.46   2.    71.  ]
 [ 36.   134.01   1.    45.  ]
 [ 40.   176.86   0.     4.  ]
 [ 28.   164.66   1.    36.  ]
 [ 28.    53.58   0.    70.  ]
 [ 41.   180.66   1.    31.  ]
 [ 70.   117.08   2.    19.  ]
 [ 53.   165.34   2.    61.  ]
 [ 57.   181.3    0.    54.  ]
 [ 41.    77.24   0.    39.  ]
 [ 20.    39.81   0.    19.  ]
 [ 39.    61.03   1.    39.  ]
 [ 70.    96.88   1.    67.  ]
 [ 19.   167.24   0.    45.  ]
 [ 41.   174.93   2.    13.  ]
 [ 61.    21.25   1.    58.  ]
 [ 47.   111.93   1.    20.  ]
 [ 55.    95.13   1.    61.  ]
 [ 19.    59.98   0.    39.  ]
 [ 77.    41.58   0.     1.  ]
 [ 38.    80.77   0.     3.  ]
 [ 50.   189.72   0.    62.  ]
 [ 29.    7

Normalizing the data

In [193]:
x_train = (x_train - x_train.mean()) / x_train.std()
x_val = (x_val - x_val.mean()) / x_val.std()
x_test = (x_test - x_test.mean()) / x_test.std()

print("Normalized Training data x : ", x_train.shape)
print(x_train)
print("Normalized Validation data x : ", x_val.shape)
print(x_val)
print("Normalized Test data x : ", x_test.shape)
print(x_test)

Normalized Training data x :  (120, 4)
[[ 0.14939985  2.75140074 -0.99469602 -0.44307837]
 [ 0.41499354  0.33061647 -0.97426573  0.35370269]
 [-0.05490298  0.92309468 -0.97426573 -0.8925446 ]
 [-0.34092695  2.19242819 -0.97426573 -0.4022178 ]
 [ 0.23112099  0.25522872 -0.95383545 -0.23877553]
 [-0.48393893 -0.30292662 -0.99469602 -0.21834525]
 [ 0.59886609  0.47955323 -0.95383545 -0.83125375]
 [-0.21834525  0.00679647 -0.95383545  0.3332724 ]
 [ 0.14939985  2.83291757 -0.97426573 -0.6473812 ]
 [ 0.53757524  2.38569867 -0.95383545  0.4558541 ]
 [-0.25920581  1.74316626 -0.97426573 -0.07533326]
 [-0.17748468  2.6186039  -0.99469602 -0.91297488]
 [-0.42264808  2.36935444 -0.97426573 -0.25920581]
 [-0.42264808  0.09995857 -0.99469602  0.43542382]
 [-0.1570544   2.69623897 -0.97426573 -0.36135723]
 [ 0.43542382  1.39728156 -0.95383545 -0.60652063]
 [ 0.088109    2.38324703 -0.95383545  0.25155127]
 [ 0.16983014  2.70931436 -0.99469602  0.10853929]
 [-0.1570544   0.58333907 -0.99469602 -0.19

Tuning the hyperparameters

In [215]:
np.random.seed(0)
w = np.random.rand(x_train.shape[1], 3)
b = np.random.rand(1, 3)
print("Initial weights : ", w)
print("Initial bias : ", b)

best_alpha = None
best_val_error = float('inf')
epochs = 1000   
possible_alphas = [0.0001, 0.001, 0.01, 0.1]

for alpha in possible_alphas:
    for epoch in range(epochs):
        z = np.dot(x_train, w) + b
        y_pred = np.exp(z) / np.sum(np.exp(z), axis=1, keepdims=True)
        error = y_train[:, np.newaxis] - y_pred
        w = w + alpha * np.dot(x_train.T, error) / len(x_train)
        b = b + alpha * np.mean(error, axis=0, keepdims=True)
        
        z_val = np.dot(x_val, w) + b
        y_pred_val = np.exp(z_val) / np.sum(np.exp(z_val), axis=1, keepdims=True)
        val_error = np.mean(np.abs(y_val[:, np.newaxis] - y_pred_val))
        
        if val_error < best_val_error:
            best_val_error = val_error
            best_alpha = alpha
            
print("Best alpha : ", best_alpha)
print("Best validation error : ", best_val_error)

Initial weights :  [[0.5488135  0.71518937 0.60276338]
 [0.54488318 0.4236548  0.64589411]
 [0.43758721 0.891773   0.96366276]
 [0.38344152 0.79172504 0.52889492]]
Initial bias :  [[0.56804456 0.92559664 0.07103606]]
Best alpha :  0.0001
Best validation error :  0.8916666666666664


Defining One Vs All

In [ ]:
def one_vs_all_logistic_regression(x_train, y_train, x_val, y_val, num_classes, alpha, epochs):
    m, n = x_train.shape
    w = np.zeros((num_classes, n))
    b = np.zeros((num_classes, 1))
    
    for c in range(num_classes):
        y_binary = (y_train == c).astype(int)
        
        for epoch in range(epochs):
            z = np.dot(x_train, w[c]) + b[c]
            y_pred = sigmoid(z)
            error = y_binary - y_pred
            
            w[c] += alpha * np.dot(x_train.T, error) / m
            b[c] += alpha * np.mean(error)
    
    return w, b

In [ ]:
num_classes = 3
alpha = best_alpha 
epochs = 1000

w, b = one_vs_all_logistic_regression(x_train, y_train, x_val, y_val, num_classes, alpha, epochs)

def predict_multi_class(x, w, b):
    z = np.dot(x, w.T) + b.T
    y_pred_prob = sigmoid(z)
    return np.argmax(y_pred_prob, axis=1)

y_pred_test_multi = predict_multi_class(x_test, w, b)

accuracy_multi = np.mean(y_test == y_pred_test_multi)

print("Accuracy (multi-class):", accuracy_multi)

confusion_matrix_multi = np.zeros((num_classes, num_classes), dtype=int)

for i in range(len(y_test)):
    confusion_matrix_multi[int(y_test[i])][int(y_pred_test_multi[i])] += 1

print("Confusion matrix (multi-class):")
print(confusion_matrix_multi)

Accuracy (multi-class): 0.575
Confusion matrix (multi-class):
[[ 5  0 16]
 [ 0  0  1]
 [ 0  0 18]]
